# EvalML Fraud Detection Demo:
This demo showcases using EvalMl to optimize models using a custom objective to predict realized business value. The goal of the model would be to take in credit card transaction data and decide whether the transaction is fraudulent. 

Data: https://www.kaggle.com/c/ieee-fraud-detection/

In [1]:
import os

import evalml
import featuretools as ft
import numpy as np
import pandas as pd

In [ ]:
train_identity = pd.read_csv('https://featuretools-static.s3.amazonaws.com/evalml/IEEE-CIS+Fraud+Detection/train_identity.csv')
train_transaction = pd.read_csv('https://featuretools-static.s3.amazonaws.com/evalml/IEEE-CIS+Fraud+Detection/train_transaction.csv')

In [ ]:
display(train_identity.head())
display(train_transaction.head())

## Merge dataframes:

Since the data given is a one-to-one relationship between the identity and transaction data, we can merge the two dataframes on the `TransactionID` column.

In [ ]:
train_df = train_transaction.merge(train_identity)

train_sample = train_df.sample(frac=0.1, random_state=1)
X_train = train_sample.drop('isFraud', axis=1)
y_train = train_sample['isFraud']

## Encode Categorical Variables:
As categorical variables are not compatible with some machine learning models, here we encode them into numerical variables by creating dummy variables.

In [ ]:
cat_cols = X_train.select_dtypes(include=['object']).columns

In [ ]:
# encode categorical features
X_train = pd.get_dummies(X_train, columns=cat_cols)

In [ ]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X_train, y_train, test_size=.8, random_state=0)

## Model Training With AUC
Here we utilize a traditional classification objective function to automatically learn the best model. Further down, 

In [ ]:
clf = evalml.AutoClassifier(objective="AUC",
                            max_pipelines=10)

### After fitting our models, we can display the rankings of all the models and also score the holdout data with the best model

In [ ]:
# fit using autoclassfier
clf.fit(X_train, y_train)

In [ ]:
clf.rankings

In [ ]:
pipeline = clf.best_pipeline
print("Model Score: {}".format(pipeline.score(X_holdout, y_holdout)))

## Custom Objective:

Here we utilize a custom objective function built within EvalML for fraud detection. Using it we can define how the model will train to provide the most realized business value. We define below that `50%` of our customers will retry a declined transaction, we earn `2%` of each transaction and we will not be able to colelct `75%` of all fraudulent transactions. Thus, the model chosen will best fit our business needs.

In [ ]:
fraud_objective = evalml.objectives.FraudDetection(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='TransactionAmt'  # column in data that contains the amount of the transaction
)

clf_fraud = evalml.AutoClassifier(objective=fraud_objective,
                            max_pipelines=10)

In [ ]:
%%time
# fit using autoclassfier
clf_fraud.fit(X_train, y_train)

### Again we can rank our models and see the performance on our holdout sets. However, this time we will see the predicted amount of dollars lost due to fraudulent transactions!

In [ ]:
clf_fraud.rankings

In [ ]:
pipeline = clf_fraud.best_pipeline
print("Best Model Dollars Lost: {}".format(pipeline.score(X_holdout, y_holdout)))

### In comparison, the model that optimized for AUC performed TODO: HOW IT PERFORMED

In [ ]:
pipeline = best_pipeline
print("AUC Model Dollars Lost: {}".format(pipeline.score(X_holdout, y_holdout, other_objectives=fraud_objective)[1]))